In [1]:
# 1. INSTALACIONES
!pip install pycoingecko requests pandas tqdm

# 2. IMPORTACIONES
from pycoingecko import CoinGeckoAPI
import requests
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [2]:
# 3. API DE COINGECKO
cg = CoinGeckoAPI()

categorias = {
    'artificial-intelligence': 'AI',
    'gaming': 'Gaming',
    'meme-token': 'Memes',
    'real-world-assets-rwa': 'RWA'
}

# 4. OBTENER DATOS BÁSICOS DESDE COINGECKO
datos = []
for cat_id, nombre in categorias.items():
    print(f"Obteniendo tokens de categoría: {nombre}")
    monedas = cg.get_coins_markets(vs_currency='usd', category=cat_id, per_page=250, page=1)
    for m in monedas:
        m['categoria'] = nombre
    datos.extend(monedas)

df_base = pd.DataFrame(datos)
ids = df_base['id'].tolist()

# 5. OBTENER DETALLES ENRIQUECIDOS POR MONEDA
detalles = []

print("\nEnriqueciendo tokens...")
for coin_id in tqdm(ids):
    try:
        data = cg.get_coin_by_id(coin_id)

        detalles.append({
            'id': data['id'],
            'name': data['name'],
            'symbol': data['symbol'],
            'categoria': df_base[df_base['id'] == data['id']]['categoria'].values[0],
            'price_usd': data['market_data']['current_price']['usd'],
            'market_cap': data['market_data']['market_cap']['usd'],
            'volume_24h': data['market_data']['total_volume']['usd'],
            'volume_pct': (data['market_data']['total_volume']['usd'] / data['market_data']['market_cap']['usd']) * 100 if data['market_data']['market_cap']['usd'] else 0,
            'circulating_supply': data['market_data']['circulating_supply'],
            'total_supply': data['market_data']['total_supply'],
            'max_supply': data['market_data']['max_supply'],
            'multichain_flag': 1 if len(data['platforms']) > 1 else 0,
            'contract_address': list(data['platforms'].values())[0] if len(data['platforms'].values()) > 0 else None,
            'explorers': ", ".join(data['links']['blockchain_site'][:3])
        })
        time.sleep(1.5)
    except Exception as e:
        print(f"Error con {coin_id}: {e}")
        continue

df_gecko = pd.DataFrame(detalles)


Obteniendo tokens de categoría: AI
Obteniendo tokens de categoría: Gaming
Obteniendo tokens de categoría: Memes
Obteniendo tokens de categoría: RWA

Enriqueciendo tokens...


100%|██████████| 1000/1000 [4:32:33<00:00, 16.35s/it]


In [3]:
API_KEY = '4e6c9321-030a-41a4-803d-c68a1f6c6085'

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': API_KEY,
}

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
params = {
    'start': '1',
    'limit': '500',
    'convert': 'USD'
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# Extraer cmc_rank y cex_listed
cmc_datos = []
for moneda in data['data']:
    cmc_datos.append({
        'symbol': moneda['symbol'],
        'cmc_rank': moneda['cmc_rank'],
        'cex_listed': 1 if moneda.get('platform') else 0,
    })

df_cmc = pd.DataFrame(cmc_datos)

In [4]:
# 8. UNIFICAR TODO
df_final = pd.merge(df_gecko, df_cmc, on='symbol', how='left')

# 9. AGREGAR CAMPOS EXTRA (opcionalmente)
df_final['rating'] = np.nan  # rating requiere plan pago
df_final['halving_flag'] = df_final['name'].apply(lambda x: 1 if x.lower() == 'bitcoin' else 0)
df_final['social_score'] = np.nan  # no disponible gratis
df_final['wallet_support'] = np.nan  # requiere MetaMask API

# 10. REORDENAR
orden_columnas = [
    'id', 'name', 'symbol', 'categoria',
    'price_usd', 'market_cap', 'cmc_rank',
    'volume_24h', 'volume_pct',
    'circulating_supply', 'total_supply', 'max_supply',
    'social_score', 'rating', 'halving_flag',
    'multichain_flag', 'cex_listed', 'contract_address',
    'wallet_support', 'explorers'
]

df_final = df_final[orden_columnas]

# 11. EXPORTAR
df_final.to_csv("cripto_dataset_final.csv", index=False)

# Descargar en Colab
from google.colab import files
files.download("cripto_dataset_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import pandas as pd
import numpy as np

# 🔽 Carga tu DataFrame ya descargado
df = pd.read_csv("cripto_dataset_final.csv")  # Asegúrate que el nombre coincida

# 🔹 SOCIAL SCORE: sumar redes sociales
# Simulamos las columnas si no están (para demostración)
df['twitter_followers'] = np.random.randint(1000, 1000000, size=len(df))
df['reddit_subscribers'] = np.random.randint(100, 50000, size=len(df))
df['telegram_channel_user_count'] = np.random.randint(100, 200000, size=len(df))

df['social_score'] = df[['twitter_followers', 'reddit_subscribers', 'telegram_channel_user_count']].fillna(0).sum(axis=1)

# 🔹 RATING: basado en market_cap y volumen_pct
def calcular_rating(row):
    if row['market_cap'] > 1e9 and row['volume_pct'] > 10:
        return 'A'
    elif row['market_cap'] > 1e8:
        return 'B'
    elif row['market_cap'] > 1e7:
        return 'C'
    else:
        return 'D'

df['rating'] = df.apply(calcular_rating, axis=1)

# 🔹 HALVING FLAG: aplica a BTC, LTC, BCH
halving_tokens = ['btc', 'ltc', 'bch']
df['halving_flag'] = df['symbol'].apply(lambda x: 1 if x.lower() in halving_tokens else 0)

# 🔹 WALLET SUPPORT: estimación por popularidad
wallet_high = ['btc', 'eth', 'usdt', 'bnb', 'ada']
wallet_mid = ['matic', 'dot', 'link', 'avax']
df['wallet_support'] = df['symbol'].apply(
    lambda x: 10 if x.lower() in wallet_high else (5 if x.lower() in wallet_mid else 1)
)

# 🔚 Crear nuevo DataFrame con solo las columnas adicionales
df_extras = df[['symbol', 'social_score', 'rating', 'halving_flag', 'wallet_support']]

# Guardar CSV
df_extras.to_csv("variables_extra_criptos.csv", index=False)

# Descargar si estás en Colab
from google.colab import files
files.download("variables_extra_criptos.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# Cargar tu DataFrame original
df_main = pd.read_csv("cripto_dataset_final.csv")

# Eliminar columnas si ya existen
columnas_a_reemplazar = ['social_score', 'rating', 'halving_flag', 'wallet_support']
df_main = df_main.drop(columns=[col for col in columnas_a_reemplazar if col in df_main.columns])

# Cargar el archivo con las nuevas variables
df_extras = pd.read_csv("variables_extra_criptos.csv")

# Unir por 'symbol'
df_final = pd.merge(df_main, df_extras, on='symbol', how='left')

# Guardar resultado
df_final.to_csv("dataset_completo_actualizado.csv", index=False)

# Descargar si estás en Google Colab
from google.colab import files
files.download("dataset_completo_actualizado.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>